#        Did financial crisis 2008-2009 impact on EU firms' capital structures?

### This document intends to provide source codes in order to replicate the analysis with different dataset
Owner: Trang Ton

Dataset used in this study is from Osiris Database. Due to different data sources, data cleaning steps might be different.

The regression analysis is not comprehensive and might ignore other financial factors that could impact the firms' capital structures. However, I hope this would give you prior ideas for further research.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from docx import Document
from statsmodels.iolib.summary2 import summary_col
from linearmodels.panel import PanelOLS # using for panel regression
from linearmodels.panel import compare # creating combined table of results - panel analysis

In [ ]:
DATA_FILE = '# Data path'
df = pd.read_csv(DATA_FILE, encoding = 'input file encoding')
df.head()

# 1. Transforming into panel data

The dataset is extracted from Osiris database. Afer exporting raw data from Osiris, transforming into panel setting.

In [ ]:
### Change column names before transforming

# Loop to drop 'm EUR' in variable names
COL = list(df.columns)
dic = dict()
label = ' m EUR '
for col in COL:
    if label in col:
        new_name = col.replace(label, '')
        dic[col] = new_name
df = df.rename(columns = dic)

# Loop to drop ' % ' in variable names
COL = list(df.columns)
dic = dict()
label = ' % '
for col in COL:
    if label in col:
        new_name = col.replace(label, '')
        dic[col] = new_name
df = df.rename(columns = dic)


# Drop columns with 'Last avail. yr' terms
COL = list(df.columns)
label_drop = 'Last avail. yr'
for col in COL:
    if label_drop in col:
        df = df.drop(columns = col)
        
# Rename Number of employees columns
COL = list(df.columns)
dic = dict()
label = '.1'
for col in COL:
    if label in col:
        new_name = col.replace(label, '')
        dic[col] = new_name
df = df.rename(columns = dic)

# Rename Number of employees columns
COL = list(df.columns)
dic = dict()
label = 'Number of employees  '
for col in COL:
    if label in col:
        new_name = col.replace(label, 'Number of employees')
        dic[col] = new_name
df = df.rename(columns = dic)

#remove duplicate columns
duplicated_columns = df.columns.duplicated()

df = df.loc[:, ~duplicated_columns]

df.head()


In [ ]:
### Transforming the data set inro panel setting

year_arr = list(range(2004, 2013))
year_arr

COL = list(df.columns)

df_final = pd.DataFrame()
df_array = []
for year in year_arr:
    
    df_yearly = pd.DataFrame()
    df_yearly['Company'] = df['Company name']
    df_yearly['Year'] = year
    df_yearly['Country'] = df['Country - (address of incorp.)']
    df_yearly['GICS'] = df['GICS code']
    df_yearly['GICS Description'] = df['GICS description']
    
    for col in COL:
        if (f'{year}') in col:
            var = col.replace(f'{year}', '')
            df_yearly[var] = df[col]
            
    df_final = pd.concat([df_final, df_yearly])

In [ ]:
### drop duplicated column Total Assets

df_final=df_final.drop(columns='Total Assets')

### Sort dataframe by company names and years
df_sort = df_final.sort_values(by=['Company', 'Year'])

In [ ]:
### Export Panel dataset and save as csv file

df_sort.to_csv('raw_EU27_panel.csv', index=False)

# 2. Data Preparation

### Applying cleaning functions, categorizing functions

1. Cleaning functions: Replace missing value, modifying data types, classify industries, etc.

In [ ]:
# Listing all used variables

EU19_list = ['AUSTRIA','BELGIUM','CYPRUS','ESTONIA','FINLAND',
             'FRANCE','GERMANY','GREECE','IRELAND','ITALY','LATVIA','LITHUANIA',
             'LUXEMBOURG','MALTA','NETHERLANDS','PORTUGAL','SLOVAKIA','SLOVENIA','SPAIN']

var_list = ['Total assets', 'Net sales','Total liabilities',
            'Market Cap.', 'ROA', 'Net debt',
           'Number of employees', 'Intangible assets']
non_missing_col = ['Total assets', 'Net sales','Total liabilities', 'Market Cap.']
positive_col = ['Total assets', 'Net sales','Total liabilities']
non_zero_col = ['Total assets', 'Net sales','Total liabilities', 'Market Cap.']
excluded_industries = ['Financials']

dtype_dic = {'Total assets': 'int64',
             'Net sales': 'int64',
             'Total liabilities': 'int64',
            'Market Cap.': 'int64'}

rename_dic = {'Return on assets (ROA)': 'ROA',
             'Number of employees ': 'Number of employees'}



In [ ]:
# Function_1 to clean missing values and change data types

def dataframe_cleaning(df, non_missing_col, positive_col, non_zero_col, dtype_dic):
    # missing value filter
    df = df.dropna(subset = non_missing_col)
    
    # dropping duplicate values
    df = df.drop_duplicates(subset = ['Company', 'Year'])
    
    # fix datatypes
    df = df.astype(dtype_dic)
    
    # filter for insolvency
    for col in positive_col:
        df = df.loc[df[col] > 0]
        
    #filter for non zero values
    for col in non_zero_col:
        df = df.loc[df[col] != 0]
        
    return df
    

In [ ]:
### Function_2 to classify industry

industry_dic = {'Energy': 10,
               'Materials': 15,
               'Industrials': 20,
               'Consumer Discretionary': 25,
               'Consumer Staples': 30,
               'Health Care': 35,
               'Financials': 40,
               'Information Technology': 45,
               'Communication Services': 50,
               'Utilities': 55,
               'Real Estate': 60}
def get_key_from_value(dictionary, value):
    for key in dictionary:
        if dictionary[key] == value:
            return key
    # If the value is not found, return None or raise an exception
    return None
def industry_classifier (GICS):
    industry = get_key_from_value(industry_dic,int(str(GICS)[:2]))
    return industry

In [ ]:
### Function_3 to filter industries

def sample_filter(df, industry_col, excluded_industries):
    # filter for financial firms
    df = df.loc[~df[industry_col].isin(excluded_industries)]

    return df

In [ ]:
### Function_4: Winsorization 

def winsorization(df, variables, lower_thresh = 0.01, upper_thresh = 0.99, by = 'Year', suffix = '_w'):
    dfs = []

    categories = df[by].unique()

    for cat in categories:
        df_cat = df.loc[df[by] == cat]

        for var in variables:
            df_cat[f'{var}{suffix}'] = df_cat[var].clip(lower = df_cat[var].dropna().quantile(lower_thresh),
                                                        upper = df_cat[var].dropna().quantile(upper_thresh))
        dfs.append(df_cat)

    df_output = pd.concat(dfs)
    
    return df_output

 2. Applying functions: import panel data, applying cleaning processes

In [ ]:
# Import new raw panel
DATA_FILE = '/Users/tonnunhatrang/Desktop/Data analysis/Python/Essay_AMA3/EU27_raw_detailed/raw_EU27_panel.csv'
df = pd.read_csv(DATA_FILE, encoding = 'utf-8')
df.head()

In [ ]:
### Applying cleaning processes
df = df.rename(columns = rename_dic)

df_clean = dataframe_cleaning(df, non_missing_col, positive_col, non_zero_col, dtype_dic)

# Apply function2 to dataframe, and create new column names Industry

df_clean['Industry'] = df['GICS'].apply(industry_classifier)

### Apply sample filter industries

df_clean = sample_filter(df_clean, industry_col = 'Industry', excluded_industries = excluded_industries)
df_clean

### Filtering Eurozone countries

EU19_data = df_clean[df_clean['Country'].isin(EU19_list)]
EU19_data

In [ ]:
EU19_data[non_missing_col].isna().sum()

In [ ]:
EU19_data[var_list].isna().sum()

In [ ]:
EU19_data = EU19_data.replace(['na', 'nan', 'N/A', 'NA', 'NaN'], '')

In [ ]:
### Replace missing values by Mean or zero (using for variables with small number of missing values compared to total obs)
values = {'ROA': EU19_data['ROA'].mean(),
         'Number of employees': 0,
         'Intangible assets': EU19_data['Intangible assets'].mean()}

df_replace_missing = EU19_data.fillna(value = values)
EU19_data = df_replace_missing
EU19_data[var_list].isna().sum()

## 4. Regression Analysis

### 1. Calculations

In [ ]:
### Control variables

df = EU19_data

df['Net_assets'] = df['Total assets'] - df['Total liabilities']

### Firm size
df['log_Firm_size'] = np.log(df['Net sales'])

### Tangibility
df['Tangible_assets'] = df['Total assets'] - df['Intangible assets']
df['Tangibility'] = df['Tangible_assets'] / df['Total assets']

### Log transformmation of Market capitalization
df['log_MarketCap'] = np.log(df['Market Cap.'])

### Add constant parameter for regression estimation
df['const'] = 1


In [ ]:
### Cleaning zero values after calucation

df = df.loc[df['Net_assets'] != 0]

In [ ]:
### Leverage calculations

### Leverage ratio : calculated by Total liabilities / Total assets

df['Total_Leverage'] = df['Total liabilities'] / df['Total assets']

### Debt ratio: calculated by Net debt/ Net assets
df['Debt_leverage'] = df['Net debt'] / df['Net_assets']

Leverage_variables = ['Total_Leverage', 'Debt_leverage']

In [ ]:
df[['Debt_leverage', 'Net debt', 'Net_assets', 'Tangibility']].isna().sum()

In [ ]:
### treating outlier by winzorization
winzorized_var = ['log_MarketCap', 'ROA', 'log_Firm_size', 'Tangibility', 'Total_Leverage', 'Debt_leverage']
df_w = winsorization(df, variables = winzorized_var, by = 'Year')

### 2. Statistic description and visualization

In [ ]:
df_industry = df['Industry'].unique()
Industry_data = pd.DataFrame()
num_company = []
for industry in df_industry:
    num = df_w.loc[df['Industry'] == f'{industry}']['Company'].nunique()
    num_company.append(num)
Industry_data['Industry'] =  df_industry
Industry_data['Number of Comapny'] = num_company
Industry_data

In [ ]:
### Descriptive statistics table:

descriptive_table = df_w.groupby('Year').agg({'Total_Leverage_w': ['mean', 'std'], 'Debt_leverage_w': ['mean', 'std'],
                                              'log_MarketCap_w': ['mean', 'std'], 'ROA_w': ['mean', 'std'],
                                          'log_Firm_size_w': ['mean', 'std'], 'Tangibility_w': ['mean', 'std']})


descriptive_table

In [ ]:
OUTPUT_DIR = ''
OUTPUT_FILE_NAME = 'EU19_Descriptive_Table.xlsx'
descriptive_table.to_excel(f'{OUTPUT_DIR}{OUTPUT_FILE_NAME}', sheet_name = 'EU19 Stats')

In [ ]:
years = np.arange(2004, 2013)
debt_leverage = pd.Series(index=years)

for year in years:
    debt_leverage[year] = df_w.loc[df_w['Year'] == year, 'Debt_leverage_w'].median()
    
    
x = years


fig, ax = plt.subplots()
ax.plot(x, debt_leverage, 'b-', linewidth=2, label = 'Debt Leverage')

ax.set_xlabel('Year')
ax.set_ylabel('Ratio')


plt.show()

In [ ]:
control_var = ['log_MarketCap_w', 'ROA_w', 'log_Firm_size_w', 'Tangibility_w']
corr_matrix = df_w[['Total_Leverage_w'] + ['Debt_leverage_w']+ control_var].corr()
corr_matrix

In [ ]:
corr_list = ['Total_Leverage_w','Debt_leverage_w','log_MarketCap_w', 'ROA_w', 'log_Firm_size_w', 'Tangibility_w']
df_corr = df_w[corr_list]

plt.figure(figsize = (5,4))
sns.heatmap(df_corr.corr())

In [ ]:
### How many companies are left (after cleaning missing values for main variables, and excluding financial industries)

unique_com = df_w['Company'].nunique()
unique_com


In [ ]:
### How many years:
unique_yr = df_w['Year'].unique()
unique_yr

In [ ]:
### which countries are included:
unique_country = sorted(EU19_data['Country'].unique())
unique_country

In [ ]:
### what industries ar included:
industries = sorted(EU19_data['Industry'].unique())
industries

### 3. Regression analysis

### 1. OLS with time-fixed effect

In [ ]:
### Create time period dummies

Pre_crisis_yr = range(2004, 2007)
Crisis_yr = range(2007, 2010)
Post_crisis_yr = range(2010, 2013)

df_w['Pre_crisis_dummy'] = 0
df_w['Crisis_dummy'] = 0
df_w['Post_crisis_dummy'] = 0

for year in year_arr:
    if year in Pre_crisis_yr:
        df_w.loc[df_w['Year'] == year, 'Pre_crisis_dummy'] = 1
    elif year in Crisis_yr:
        df_w.loc[df_w['Year'] == year, 'Crisis_dummy'] = 1
    elif year in Post_crisis_yr:
        df_w.loc[df_w['Year'] == year, 'Post_crisis_dummy'] = 1

In [ ]:
df_w[['Year', 'Pre_crisis_dummy', 'Crisis_dummy', 'Post_crisis_dummy']]

In [ ]:
### Linear regrssion with time period dummies, taking Crisis_dummy as reference time

## Case 1: 'Total leverage' as outcome variable

X = ['log_MarketCap_w', 'ROA_w', 'log_Firm_size_w', 'Tangibility_w',
    'Pre_crisis_dummy', 'Post_crisis_dummy', 'const']

Y = 'Total_Leverage_w'


reg1 = sm.OLS(endog = df_w[Y],
             exog = df_w[X],
             missing = 'drop')

results1 = reg1.fit().get_robustcov_results(cov_type = 'HC0')

print(results1.summary())

In [ ]:
## Case 2: 'Debt_leverage' as outcome variable

x = ['log_MarketCap_w', 'ROA_w', 'log_Firm_size_w', 'Tangibility_w',
    'Pre_crisis_dummy', 'Post_crisis_dummy', 'const']

y = 'Debt_leverage_w'



reg2 = sm.OLS(endog = df_w[y],
             exog = df_w[x],
             missing = 'drop')

results2 = reg2.fit().get_robustcov_results(cov_type = 'HC0')

print(results2.summary())

In [ ]:
### Print linear regression results

results_file = summary_col([results1,results2],stars=True)


print(results_file)


### 2. Panel regression

In [ ]:
### Setting panel data

df_w.set_index(['Company', 'Year'], inplace=True)
df_w.head()

#### Total leverage analysis

In [ ]:
### Panel regression: with time fixed effect - Period dummies

x_1 = ['log_MarketCap_w', 'ROA_w', 'log_Firm_size_w', 'Tangibility_w', 'const']

y_1 = 'Total_Leverage_w'

period_dummies_1 = ['Pre_crisis_dummy', 'Post_crisis_dummy']

panel_model_1 = PanelOLS.from_formula(y_1 + ' ~ ' + ' + '.join(x_1) +' + ' + ' + '.join(period_dummies_1), df_w, check_rank=False)

results3 = panel_model_1.fit(cov_type = 'robust')

print(results3.summary)


In [ ]:
### Panel regression: with interaction term - Post_crisis * ROA

period_dummies_2 = ['Crisis_dummy', 'Post_crisis_dummy']

panel_model_2 = PanelOLS.from_formula(y_1 + ' ~ ' + ' + '.join(x_1) +' + ' + ' + '.join(period_dummies_2) + '+ ROA_w * Crisis_dummy' + ' + ROA_w * Post_crisis_dummy', df_w, check_rank=False)

results4 = panel_model_2.fit(cov_type = 'robust')

print(results4.summary)



#### Debt Leverage analysis

In [ ]:
### Panel regression: with time fixed effect - Period dummies

x_1 = ['log_MarketCap_w', 'ROA_w', 'log_Firm_size_w', 'Tangibility_w', 'const']

period_dummies_1 = ['Pre_crisis_dummy', 'Post_crisis_dummy']

panel_model_3 = PanelOLS.from_formula(y + ' ~ ' + ' + '.join(x_1) +' + ' + ' + '.join(period_dummies_1), df_w, check_rank=False)

results5 = panel_model_3.fit(cov_type = 'robust')

print(results5.summary)



In [ ]:
### Panel regression: with interaction term - Post_crisis * ROA

period_dummies_2 = ['Crisis_dummy', 'Post_crisis_dummy']

panel_model_4 = PanelOLS.from_formula(y + ' ~ ' + ' + '.join(x_1) +' + ' + ' + '.join(period_dummies_2) + '+ ROA_w * Crisis_dummy' + ' + ROA_w * Post_crisis_dummy', df_w, check_rank=False)

results6 = panel_model_4.fit(cov_type = 'robust')

print(results6.summary)



In [ ]:
### Print panel regression results

summary_table = compare({'panel_model_1': results3, 'panel_model_2': results4,
                         'panel_model_3': results5, 'panel_model_4': results6},stars=True)
print(summary_table)